# Config

In [1]:
COUNT_FAILED_ORDERS = True

# Imports

In [2]:
import pandas as pd

import sys
sys.path.append("..") #There are better ways to do this

from dragon_fruit.calculation_functions.CalculateFeatures import calculate_time_between_orders
from dragon_fruit.calculation_functions.HelperFunctions import calculate_cyclic_encoding

pd.set_option('display.max_columns', None)

# Reading Data

In [3]:
Data = pd.read_csv("../data/machine_learning_challenge_order_data.csv.gz")
Data

,customer_id,order_date,order_hour,customer_order_rank,is_failed,voucher_amount,delivery_fee,amount_paid,restaurant_id,city_id,payment_id,platform_id,transmission_id
0,000097eabfd9,2015-06-20,19,1.0,0,0.0,0.000,11.46960,5803498,20326,1779,30231,4356
1,0000e2c6d9be,2016-01-29,20,1.0,0,0.0,0.000,9.55800,239303498,76547,1619,30359,4356
2,000133bb597f,2017-02-26,19,1.0,0,0.0,0.493,5.93658,206463498,33833,1619,30359,4324
3,00018269939b,2017-02-05,17,1.0,0,0.0,0.493,9.82350,36613498,99315,1619,30359,4356
4,0001a00468a6,2015-08-04,19,1.0,0,0.0,0.493,5.15070,225853498,16456,1619,29463,4356
...,...,...,...,...,...,...,...,...,...,...,...,...,...
786595,fffe9d5a8d41,2016-09-30,20,NaN,1,0.0,0.000,10.72620,983498,10346,1779,29463,212
786596,ffff347c3cfa,2016-08-17,21,1.0,0,0.0,0.000,7.59330,52893498,41978,1619,30359,4356
786597,ffff347c3cfa,2016-09-15,21,2.0,0,0.0,0.000,5.94720,164653498,41978,1619,30359,4356
786598,ffff4519b52d,2016-04-02,19,1.0,0,0.0,0.000,21.77100,16363498,80562,1491,29751,4228


# Feature Engineering and Data Preparation

## Data cleaning

In [4]:
print("As revealed in the EDA, exactly %d records had an invalid outliers date before the beginning date." % len(Data[Data.order_date<'2015-03-01']))
print("These are going to be discarded.")

As revealed in the EDA, exactly 53 records had an invalid outliers date before the beginning date.
These are going to be discarded.


In [5]:
Data = Data[Data.order_date>= '2015-03-01']

In [6]:
print("There is exactly %d duplicated records. They will be dropped." %len(Data[Data.duplicated()]))

There is exactly 546 duplicated records. They will be dropped.


In [7]:
Data = Data.drop_duplicates()

### Historical feature
Let's add at each order the time since the last order, this could be a helpful feature.
(Since this feature shares a lot of similarity and calculations with the label/target feature, I'll integrate it into the same function)

In [8]:
Data = calculate_time_between_orders(Data, COUNT_FAILED_ORDERS)
Data

,customer_id,order_date,order_hour,is_failed,voucher_amount,delivery_fee,amount_paid,restaurant_id,city_id,payment_id,platform_id,transmission_id,order_time,time_since_last_order,customer_order_rank,time_to_next_order,is_returning_customer
0,000097eabfd9,2015-06-20,19,0,0.0,0.000,11.46960,5803498,20326,1779,30231,4356,2015-06-20 19:00:00,NaN,1,NaN,0
1,0000e2c6d9be,2016-01-29,20,0,0.0,0.000,9.55800,239303498,76547,1619,30359,4356,2016-01-29 20:00:00,NaN,1,NaN,0
2,000133bb597f,2017-02-26,19,0,0.0,0.493,5.93658,206463498,33833,1619,30359,4324,2017-02-26 19:00:00,NaN,1,NaN,0
3,00018269939b,2017-02-05,17,0,0.0,0.493,9.82350,36613498,99315,1619,30359,4356,2017-02-05 17:00:00,NaN,1,NaN,0
4,0001a00468a6,2015-08-04,19,0,0.0,0.493,5.15070,225853498,16456,1619,29463,4356,2015-08-04 19:00:00,NaN,1,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
786595,fffe9d5a8d41,2016-09-30,20,1,0.0,0.000,10.72620,983498,10346,1779,29463,212,2016-09-30 20:00:00,0.0,3,NaN,0
786596,ffff347c3cfa,2016-08-17,21,0,0.0,0.000,7.59330,52893498,41978,1619,30359,4356,2016-08-17 21:00:00,NaN,1,696.0,1
786597,ffff347c3cfa,2016-09-15,21,0,0.0,0.000,5.94720,164653498,41978,1619,30359,4356,2016-09-15 21:00:00,696.0,2,NaN,0
786598,ffff4519b52d,2016-04-02,19,0,0.0,0.000,21.77100,16363498,80562,1491,29751,4228,2016-04-02 19:00:00,NaN,1,NaN,0


## Time features

In [9]:
#As it is, we're not using the information embedded in 'order_date', so let's extract it!
Data['day_of_month'] = Data.order_time.dt.day #To capture monthly seasonality
Data['day_of_week'] = Data.order_time.dt.weekday #To capture weekly seasonality
Data['month'] = Data.order_time.dt.month #To capture within year seasonality
Data['year'] = Data.order_time.dt.year #To capture YoY growth

Now handling time features is always tricky, we lose so much information if we encode them as categorical variables, because indeed there is some ordinality there.

So to keep the ordinality information we encode them as numeric features, **BUT** there's a problem with encoding them just as they are, we lose the periodic information. For example, in hours 1 is as close to 2, as 0 is to 23!

**Solution: Use cyclic encoding.**

**Cyclic encoding** essentially plots the variable range on a circle, by projecting it using sine and cosine.

Helpful source: http://blog.davidkaleko.com/feature-engineering-cyclical-features.html

So I am going to calculate the sine and cosine of: hour, day_of_month, day_of_week, and month this way.

In [10]:
Data['hour_sin'] = calculate_cyclic_encoding(Data.order_hour, 24, 'sin')
Data['hour_cos'] = calculate_cyclic_encoding(Data.order_hour, 24, 'cos')

Data['day_of_month_sin'] = calculate_cyclic_encoding(Data.day_of_month-1, 30, 'sin') #We subtract 1 because it doesn't start from 0
Data['day_of_month_cos'] = calculate_cyclic_encoding(Data.day_of_month-1, 30, 'cos')

Data['day_of_week_sin'] = calculate_cyclic_encoding(Data.day_of_week, 7, 'sin')
Data['day_of_week_cos'] = calculate_cyclic_encoding(Data.day_of_week, 7, 'cos')

Data['month_sin'] = calculate_cyclic_encoding(Data.month-1, 12, 'sin') #We subtract 1 because it doesn't start from 0
Data['month_cos'] = calculate_cyclic_encoding(Data.month-1, 12, 'cos')

### Dropping Irrelevant Features

In [11]:
#Categories that are too many, with no extra details, don't add much information, and hence will be dropped
Data = Data.drop(['restaurant_id', 'city_id'], axis=1)

#Dropping already-processed features:
Data = Data.drop(['order_date', 'order_hour', 'day_of_month', 'day_of_week', 'month',], axis=1)


Data

,customer_id,is_failed,voucher_amount,delivery_fee,amount_paid,payment_id,platform_id,transmission_id,order_time,time_since_last_order,customer_order_rank,time_to_next_order,is_returning_customer,year,hour_sin,hour_cos,day_of_month_sin,day_of_month_cos,day_of_week_sin,day_of_week_cos,month_sin,month_cos
0,000097eabfd9,0,0.0,0.000,11.46960,1779,30231,4356,2015-06-20 19:00:00,NaN,1,NaN,0,2015,-0.965926,0.258819,-0.743145,-0.669131,-0.974928,-0.222521,0.500000,-8.660254e-01
1,0000e2c6d9be,0,0.0,0.000,9.55800,1619,30359,4356,2016-01-29 20:00:00,NaN,1,NaN,0,2016,-0.866025,0.500000,-0.406737,0.913545,-0.433884,-0.900969,0.000000,1.000000e+00
2,000133bb597f,0,0.0,0.493,5.93658,1619,30359,4324,2017-02-26 19:00:00,NaN,1,NaN,0,2017,-0.965926,0.258819,-0.866025,0.500000,-0.781831,0.623490,0.500000,8.660254e-01
3,00018269939b,0,0.0,0.493,9.82350,1619,30359,4356,2017-02-05 17:00:00,NaN,1,NaN,0,2017,-0.965926,-0.258819,0.743145,0.669131,-0.781831,0.623490,0.500000,8.660254e-01
4,0001a00468a6,0,0.0,0.493,5.15070,1619,29463,4356,2015-08-04 19:00:00,NaN,1,NaN,0,2015,-0.965926,0.258819,0.587785,0.809017,0.781831,0.623490,-0.500000,-8.660254e-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
786595,fffe9d5a8d41,1,0.0,0.000,10.72620,1779,29463,212,2016-09-30 20:00:00,0.0,3,NaN,0,2016,-0.866025,0.500000,-0.207912,0.978148,-0.433884,-0.900969,-0.866025,-5.000000e-01
786596,ffff347c3cfa,0,0.0,0.000,7.59330,1619,30359,4356,2016-08-17 21:00:00,NaN,1,696.0,1,2016,-0.707107,0.707107,-0.207912,-0.978148,0.974928,-0.222521,-0.500000,-8.660254e-01
786597,ffff347c3cfa,0,0.0,0.000,5.94720,1619,30359,4356,2016-09-15 21:00:00,696.0,2,NaN,0,2016,-0.707107,0.707107,0.207912,-0.978148,0.433884,-0.900969,-0.866025,-5.000000e-01
786598,ffff4519b52d,0,0.0,0.000,21.77100,1491,29751,4228,2016-04-02 19:00:00,NaN,1,NaN,0,2016,-0.965926,0.258819,0.207912,0.978148,-0.974928,-0.222521,1.000000,6.123234e-17


## Encoding Categorical Variables
We have three variables that are categorical in nature, but that are represented as numbers. Since there is no true ordinal relationship between them, this will be misleading to the model.

So we use One-Hot-Encoding.

In [12]:
Data = pd.get_dummies(Data, columns = ['payment_id', 'platform_id', 'transmission_id'])
Data

,customer_id,is_failed,voucher_amount,delivery_fee,amount_paid,order_time,time_since_last_order,customer_order_rank,time_to_next_order,is_returning_customer,year,hour_sin,hour_cos,day_of_month_sin,day_of_month_cos,day_of_week_sin,day_of_week_cos,month_sin,month_cos,payment_id_1491,payment_id_1523,payment_id_1619,payment_id_1779,payment_id_1811,platform_id_525,platform_id_22167,platform_id_22263,platform_id_22295,platform_id_29463,platform_id_29495,platform_id_29751,platform_id_29815,platform_id_30135,platform_id_30199,platform_id_30231,platform_id_30359,platform_id_30391,platform_id_30423,transmission_id_212,transmission_id_1988,transmission_id_2020,transmission_id_4196,transmission_id_4228,transmission_id_4260,transmission_id_4324,transmission_id_4356,transmission_id_4996,transmission_id_21124
0,000097eabfd9,0,0.0,0.000,11.46960,2015-06-20 19:00:00,NaN,1,NaN,0,2015,-0.965926,0.258819,-0.743145,-0.669131,-0.974928,-0.222521,0.500000,-8.660254e-01,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0
1,0000e2c6d9be,0,0.0,0.000,9.55800,2016-01-29 20:00:00,NaN,1,NaN,0,2016,-0.866025,0.500000,-0.406737,0.913545,-0.433884,-0.900969,0.000000,1.000000e+00,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0
2,000133bb597f,0,0.0,0.493,5.93658,2017-02-26 19:00:00,NaN,1,NaN,0,2017,-0.965926,0.258819,-0.866025,0.500000,-0.781831,0.623490,0.500000,8.660254e-01,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0
3,00018269939b,0,0.0,0.493,9.82350,2017-02-05 17:00:00,NaN,1,NaN,0,2017,-0.965926,-0.258819,0.743145,0.669131,-0.781831,0.623490,0.500000,8.660254e-01,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0
4,0001a00468a6,0,0.0,0.493,5.15070,2015-08-04 19:00:00,NaN,1,NaN,0,2015,-0.965926,0.258819,0.587785,0.809017,0.781831,0.623490,-0.500000,-8.660254e-01,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
786595,fffe9d5a8d41,1,0.0,0.000,10.72620,2016-09-30 20:00:00,0.0,3,NaN,0,2016,-0.866025,0.500000,-0.207912,0.978148,-0.433884,-0.900969,-0.866025,-5.000000e-01,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
786596,ffff347c3cfa,0,0.0,0.000,7.59330,2016-08-17 21:00:00,NaN,1,696.0,1,2016,-0.707107,0.707107,-0.207912,-0.978148,0.974928,-0.222521,-0.500000,-8.660254e-01,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0
786597,ffff347c3cfa,0,0.0,0.000,5.94720,2016-09-15 21:00:00,696.0,2,NaN,0,2016,-0.707107,0.707107,0.207912,-0.978148,0.433884,-0.900969,-0.866025,-5.000000e-01,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0
786598,ffff4519b52d,0,0.0,0.000,21.77100,2016-04-02 19:00:00,NaN,1,NaN,0,2016,-0.965926,0.258819,0.207912,0.978148,-0.974928,-0.222521,1.000000,6.123234e-17,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0


In [13]:
#If we don't count failed orders, then it could be a good idea to remove them for now
if COUNT_FAILED_ORDERS == False:
    Data = Data[Data.is_failed==0].copy()
    Data = Data.drop('is_failed',axis=1)
    

# Saving Engineered Data for Future Use

In [14]:
Data.to_csv("Engineered_Data.csv",index=False)